In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import re
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from kaggle_utils import move_feature

In [2]:
# paths_feim = glob('../output/feature_importances/20190925_1450__CV0-9581588018233685__feature2114.gz') \
# paths_feim = glob('../output/feature_importances/20190925_1633__CV0-9582357322305678__feature2763.gz') \
paths_feim = glob('../output/feature_importances/20190927_0221__CV0-9594791704263358__feature1844.gz')
# paths_feim = glob('../output/feature_importances/20190921_2310__CV0-9554268637512479__feature1142.gz')
paths_feim = glob('../output/feature_importances/20191003_0728__CV0-9564154607798349__feature943.gz')
# + glob('../output/feature_importances/20190909*.gz') \
# + glob('../output/feature_importances/20190908*.gz') 
# + glob('../output/feature_importances/20190907*.gz') \
# + glob('../output/feature_importances/20190906*.gz')
# paths_feim = glob('../output/selection_feature/*.gz')
list_good_feature = []
list_feim = []

for path in paths_feim:
    try:
        lb_score = re.search(rf'CV([^/.]*)__', path).group(1)
    except AttributeError:
        pass
    if float(lb_score.replace('-', '.'))<0.9167:
        continue

    feim = read_pkl_gzip(path)
#     feim = feim[feim['is_valid']==1].head(1).tail(1)
#     feim['score'] = lb_score
#     list_feim.append(feim)
# df_feim = pd.concat(list_feim, axis=0)
# df_feim.sort_values(by='importance', ascending=False, inplace=True)
# display(df_feim)
#     tmp_list = feim.head(1).feature.tolist()
#     for feature in tmp_list:
#         if feature not in set(list_good_feature):
#             list_good_feature.append(feature)

In [4]:
from_dir = '../submit/add_feature/'
# from_dir = '../submit/escape/'
# from_dir = '../submit/re_sub/'
# from_dir = 'valid_use'
# from_dir = 'valid_trush'
# from_dir = 'useless'
# from_dir = 'add_feature'
# to_dir = 'valid_use'
# to_dir = 'valid_trush'
to_dir = '../submit/re_sub/'
# to_dir = '../submit/add_feature/'
# for feature in df_feim['feature'].values:
# for feature in feim[feim['is_valid']==1]['feature'].values:
# feim = feim.loc[moves]
# feim = feim[feim['imp_avg']<500]
cnt = 0
for feature in feim.index:
    move_feature([feature], from_dir, to_dir)
    cnt += 1
print(cnt)

943


In [22]:
for feature in feim.index:
    if feature.count('703'):
        print(feature)

703__user_id_bear_agg_D2_max
703__user_id_bear_agg_D3_mean
703__user_id_bear_agg_C13_skew
703__user_id_bear_agg_C1_max
703__user_id_bear_agg_C14_mean
703__user_id_bear_agg_TransactionAmt_mean
703__user_id_bear_agg_D1_skew


In [19]:
COLUMN_ID = 'TransactionID'
probing = pd.read_csv('../input/20190929_probing.csv')
ids = probing[probing['Probing_isFraud']==1][COLUMN_ID].values
print(len(ids), len(set(ids)))

821 821


In [109]:
import shutil

list_from_dir = ['raw_use' , 'org_use']
list_from_dir += ['raw_trush' , 'org_trush', 'valid_trush']
to_dir = 'valid'
list_not_found = []

for feature in list_good_feature:
    for from_dir in list_from_dir:
        
        path_train_feature = f"../feature/{from_dir}/{feature}_train.gz"
        path_test_feature = f"../feature/{from_dir}/{feature}_test.gz"
        
        if os.path.exists(path_train_feature) and os.path.exists(path_test_feature):
#         if os.path.exists(path_train_feature):
#         if os.path.exists(path_test_feature):
            try:
                shutil.move(path_train_feature, f'../feature/{to_dir}')
                shutil.move(path_test_feature, f'../feature/{to_dir}')
            except FileNotFoundError:
                print(feature)
                list_not_found.append(feature)